In [6]:
import itertools
import random
import simpy

RANDOM_SEED = 42
ROOM_CLEANING_SIZE = 200       # máximo nive de limpieza de una habitación 
THRESHOLD = 25                 # mínimo de limpieza/confort (% del total)
TOURIST_ENERGY_SIZE = 50       # nivel máximo de descanso
TOURIST_ENERGY_LEVEL = [5, 25] # nivel de energía de los turistas (menor_energía => más_sueño)
SLEEP_SPEED = 2                # tasa de recuperar energía (u / second)
HOUSEMAID_TIME = 300           # tiempo que tarda la mucama en limpiar la habitación (segundos)
T_INTER = [30, 300]            # intervalo entre la llegada de los turistas
SIM_TIME = 1000                # tiempo total de la simulación

# Agents
## Tourist

In [1]:
def tourist(name, env, hotel, room):
    """A car arrives at the gas station for refueling.

    It requests one of the gas station's fuel pumps and tries to get the
    desired amount of fuel from it. If the station's fuel tank is
    depleted, the car has to wait for the tank truck to arrive.

    """
    tourist_energy_level = random.randint(*TOURIST_ENERGY_LEVEL)
    print(f'{env.now:6.1f} s: {name} arrived at hotel')
    with hotel.request() as req:
        # Request one of the rooms
        yield req

        # Get the required amount of energy
        energy_required = TOURIST_ENERGY_SIZE - tourist_energy_level
        yield room.get(energy_required)

        # The "actual" recovering of energy process takes some time
        yield env.timeout(energy_required / SLEEP_SPEED)

        print(f'{env.now:6.1f} s: {name} recovered with {energy_required:.1f}units of sleep')


## Workers

In [8]:
def housemaid(env, room):
    """Arrives at the room and finish of clean it after a certain delay."""
    yield env.timeout(HOUSEMAID_TIME)
    amount = room.capacity - room.level
    room.put(amount)
    print(
        f'{env.now:6.1f} s: Housemaid finished and the room is clean'
    )

## Manager

In [11]:
def manager(env, room):
    """Periodically check the clean level of the room and call the housemaid
       if the level falls below a threshold."""
    while True:
        if room.level / room.capacity * 100 < THRESHOLD:
            # We need to call the housemaid now!
            print(f'{env.now:6.1f} s: Housemaid is cleaning the room')
            # Wait for the tank truck to arrive and refuel the station tank
            yield env.process(housemaid(env, room))

        yield env.timeout(10)  # Check every 10 seconds

In [4]:
def tourist_generator(env, hotel, room):
    """Generate new tourists that arrive at the hotel."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(tourist(f'Tourist {i}', env, hotel, room))

# Simulation

In [12]:
# Setup and start the simulation
print('Hotel is open')
random.seed(RANDOM_SEED)

# Create environment and start processes
env = simpy.Environment()
hotel = simpy.Resource(env, 2)
room = simpy.Container(env, ROOM_CLEANING_SIZE, init=ROOM_CLEANING_SIZE)
env.process(manager(env, room))
env.process(tourist_generator(env, hotel, room))

# Execute!
env.run(until=SIM_TIME)

Hotel is open
  87.0 s: Tourist 0 arrived at hotel
 105.5 s: Tourist 0 recovered with 37.0units of sleep
 129.0 s: Tourist 1 arrived at hotel
 148.0 s: Tourist 1 recovered with 38.0units of sleep
 284.0 s: Tourist 2 arrived at hotel
 305.0 s: Tourist 2 recovered with 42.0units of sleep
 385.0 s: Tourist 3 arrived at hotel
 398.5 s: Tourist 3 recovered with 27.0units of sleep
 459.0 s: Tourist 4 arrived at hotel
 460.0 s: Housemaid is cleaning the room
 481.0 s: Tourist 4 recovered with 44.0units of sleep
 705.0 s: Tourist 5 arrived at hotel
 750.0 s: Tourist 6 arrived at hotel
 760.0 s: Housemaid finished and the room is clean
 779.0 s: Tourist 6 recovered with 38.0units of sleep
 781.5 s: Tourist 5 recovered with 43.0units of sleep
 891.0 s: Tourist 7 arrived at hotel
 904.0 s: Tourist 7 recovered with 26.0units of sleep
